# Topic Modelling with Gensim

Notebook written by Snehal Gawas - Data Scientist @ IBM

In [145]:
# Importing required libraries

In [4]:
import nltk

In [ ]:
# Install packages if needed
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('tagsets')
!pip install gensim
!pip install pyLDAvis
!pip install inflect
!pip install spacy

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/dsxuser/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/dsxuser/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /home/dsxuser/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/dsxuser/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package tagsets to /home/dsxuser/nltk_data...
[nltk_data]   Unzipping help/tagsets.zip.
    100% |████████████████████████████████| 22.6MB 38kB/s  eta 0:00:01
Requirement not upgraded as not directly required: numpy>=1.11.3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from gensim)
Requirement not upgraded as not directly required: six>=1.5.0 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from gensim)
Requireme

In [9]:
from spacy.lang.en import STOP_WORDS
import pandas as pd
import numpy as np
import requests
import json
import string
import time
import re


import matplotlib.pyplot as plt
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.tag.perceptron import PerceptronTagger
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
from gensim import corpora
import pyLDAvis.gensim
from nltk import ngrams,wordpunct_tokenize
from gensim.models import Phrases
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

In [10]:
# The code was removed by DSX for sharing.

In [21]:
temp_df = pd.read_csv(project.get_file('hn_stories.csv'))
temp_df.shape

(6959, 11)

## Missing Values

In [22]:
Data= temp_df

In [23]:
Data.isnull().sum()

by                0
descendants       1
id                0
kids              0
pain_perdu     6959
score             0
text            793
time              0
title             0
type              0
url             430
dtype: int64

In [25]:
Data.drop(['pain_perdu'],axis=1,inplace=True)

## Cleaning Data
It inlcudes removing punctuations,special charactres and symbols along with stop words

In [43]:
exclude = set(string.punctuation) 
#Technical stop words
STOP_WORDS.update(('nan','zip','function','load','model','code','file','example','print','value','given','val','variable',
             'x','y','sudo','job','figure','train','test','input','output','hash','list','return','dataset','design','install'))

#General Other stop words
STOP_WORDS.update(('email','new','jan','feb','mar','apr','may','jun','jul','aug','sept','oct','nov','dec','view','january','february',
             'march','april','june','july','august','september','october','november','december','get', 'sign', 
             'move', 'see', 'run', 'create', 'want', 'need', 'team', 'number','day','would','code',
            'add','print','image','web','us','way','problem','know','post','method','business','customer','company','event','help',
            'text','course','article','book','user','let','group','competition','country','prize','participant','submission','refresh'
            'reload','tab','window','girl','united','states','question','answer','share','people','skill','pm','better','best','select','price'
                  ,'case','datum'))

def clean_data(text):
    text = str(text)
    
#     # remove non- Ascii charactres 
    Non_ascii_free=''.join([char if ord(char) < 128 else '' for char in text])
    #remove numeric characters
    Numeric_free=''.join([char for char in Non_ascii_free if not char.isdigit()])
    #remove punctuations
    Punct_free=''.join(char for char in Numeric_free if char not in exclude)
    #remove stopwords()
    Stopwords1_free=" ".join(word for word in Punct_free.lower().split() if word not in STOP_WORDS)
#      #remove stopwords()
#     Stopwords2_free=" ".join(word for word in Stopwords1_free.lower().split() if word not in NUM_WORDS)
#      #remove stopwords()
#     Stopwords3_free=" ".join(word for word in Stopwords2_free.lower().split() if word not in ORDINAL_WORDS)
    #Remove URLs
    Url_Free=' '.join(word  for word in Stopwords1_free.lower().split() if not ((word.startswith('www') or (word.startswith('http')))))
    # Remove .pynb and .csv file names
    File_Free =' '.join(word for word in Url_Free.lower().split() if not ((word.endswith('pynb') or (word.endswith('csv') or (word.endswith('md'))))))
    #Remove longs words
    Clean =' '.join(word for word in File_Free.lower().split() if (len(word)<=20))
     #Remove short words
    Clean =' '.join(word for word in File_Free.lower().split() if (len(word)>1))
   
    return Clean
    

In [44]:
Data['text'].head()

0    © Copyright 2000 - 2016 Fox Television Station...
1    \n\n\n\n            var postLoadFunctions = {}...
2     by Matthew Hodgson | Apr 26, 2018 | General, ...
3    Uncovering the best of the web, while pointing...
4    \n \n\tCyber security has evolved into a key i...
Name: text, dtype: object

In [45]:
Data['clean_text']=Data['text'].apply(lambda x:clean_data(x))
Data.head()

,by,descendants,id,kids,score,text,time,title,type,url,clean_text
0,pain_perdu,77.0,16936533,"[16937090, 16936857, 16936689, 16936691, 16937...",141,© Copyright 2000 - 2016 Fox Television Station...,1524781744,DNA linked to Golden State Killer arrest came ...,story,http://www.ktvu.com/news/dna-linked-to-golden-...,copyright fox television stations llc rights r...
1,rezashirazian,57.0,16935270,"[16936257, 16935515, 16936338, 16937273, 16935...",234,\n\n\n\n var postLoadFunctions = {}...,1524773832,Square to buy Weebly for $365M,story,https://www.cnbc.com/2018/04/26/square-to-buy-...,var postloadfunctions var foreseeenabled var d...
2,arctux,109.0,16933736,"[16935012, 16935247, 16935094, 16934190, 16937...",402,"by Matthew Hodgson | Apr 26, 2018 | General, ...",1524762980,Matrix and Riot Confirmed as the Basis for Fra...,story,https://matrix.org/blog/2018/04/26/matrix-and-...,matthew hodgson general news press release unc...
3,ingve,102.0,16933265,"[16933598, 16933544, 16934620, 16933861, 16934...",462,"Uncovering the best of the web, while pointing...",1524759583,Magnasanti: Large and Terrifying SimCity (2010),story,https://rumorsontheinternets.org/2010/10/14/ma...,uncovering pointing laughing worst comments pi...
4,velmu,20.0,16934656,"[16935702, 16936638, 16937333, 16935952, 16937...",92,\n \n\tCyber security has evolved into a key i...,1524769885,"SEC Penalizes Yahoo $35M for Massive, Undisclo...",story,https://www.jdsupra.com/legalnews/sec-penalize...,cyber security evolved key issue media reports...


## Tokenized words  
Given a character sequence and a defined document unit, tokenization is the task of chopping it up into pieces, called tokens

In [47]:
tokenizer = RegexpTokenizer(r'\w+')
Data['Tokenized_Documents'] = Data['clean_text'].apply(lambda row: tokenizer.tokenize(row))

In [48]:
for index, row in Data.iterrows():
    temp=[]
    for i in row['Tokenized_Documents']:
        if len(i)!=1:
            temp.append(i)
    Data.set_value(index, 'Tokenized_Documents', temp)

In [49]:
# Create document list
start_time = time.time()
docs_clean=[]
docs_clean = [doc for doc in Data['Tokenized_Documents']] 
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0008847713470458984 seconds ---


In [50]:
# Add bigrams to docs (only ones that appear 5 times or more).
bigram = Phrases(docs_clean,min_count=5)
start_time = time.time()
for idx in range(len(docs_clean)):
    for token in bigram[docs_clean[idx]]:
        if '_' in token:
            while (token.split('_')[0] in docs_clean[idx]):
                docs_clean[idx].remove(token.split('_')[0])
            while (token.split('_')[1] in docs_clean[idx]):   
                docs_clean[idx].remove(token.split('_')[1])
            docs_clean[idx].append(str(token))                
print("--- %s seconds ---" % (time.time() - start_time))

--- 93.22377371788025 seconds ---


In [52]:
#Train/ Test Split
train =[]
test=[]
len(docs_clean)-5
train=docs_clean[:len(docs_clean)-10]
test=docs_clean[len(docs_clean)-10:len(docs_clean)]
print('Train Set:',len(train))
print('Test Set:', len(test))

Train Set: 6949
Test Set: 10


In [53]:
counter=0
for doc in docs_clean:
    if counter<6:
        print(Counter(doc).most_common(5))
        print('----------------------')
        counter=counter+1

[('said', 16), ('investigators', 6), ('crimes', 5), ('outside', 5), ('home', 4)]
----------------------
[('square', 9), ('weebly', 7), ('balakrishnan', 3), ('anita', 3), ('deirdre', 3)]
----------------------
[('matrix', 10), ('project', 5), ('open_source', 5), ('official', 4), ('dinsic', 4)]
----------------------
[('city', 19), ('magnasanti', 14), ('simcity', 9), ('oscala', 7), ('pingback', 6)]
----------------------
[('service', 15), ('jd_supra', 13), ('yahoo', 11), ('use', 7), ('firm', 6)]
----------------------
[('theory', 13), ('schwinger', 13), ('work', 10), ('long', 8), ('mexico', 7)]
----------------------


## Creating Dictionary
The term dictionary of our corpus, where every unique term is assigned an index.

In [54]:
start_time = time.time()
dictionary = corpora.Dictionary(train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 4.816339492797852 seconds ---


In [55]:
print(dictionary)

Dictionary(167269 unique tokens: ['kellers', 'michamus_points', 'bumblers', 'solv', 'shortsighted']...)


## Current topics in content

In [56]:
# Filter out words that occur more than 40% of the documents.
dictionary.filter_extremes(no_above=0.4)

In [57]:
dictionary.save('hn_content.dict') 

In [58]:
dictionary = corpora.Dictionary.load('hn_content.dict')

In [59]:
print(dictionary)

Dictionary(42856 unique tokens: ['kellers', 'prostitutes', 'vegetarian', 'held_evertightening', 'software_architect']...)


## Create Bag of words 
- Text (such as document) is represented as the bag (multiset) of its words
- The function doc2bow() simply counts the number of occurrences of each distinct word, converts the word to its integer word id and returns the result as a sparse vector.
- corpus is a document-term matrix and now we’re ready to generate an LDA model:

In [60]:
corpus = [dictionary.doc2bow(doc) for doc in train]

In [61]:
corpora.MmCorpus.serialize('hn_content.mm', corpus)
corpus = corpora.MmCorpus('hn_content.mm')

In [63]:
print(corpus)

MmCorpus(6949 documents, 42856 features, 1687158 non-zero entries)


## Train LDA model
- LDA assumes documents are produced from a mixture of topics
- Those topics then generate words based on their probability distribution.
- Steps:
    - Determine the number of words in a document. Let’s say our document has 6 words.
    - Determine the mixture of topics in that document. For example, the document might contain 1/2 the topic “Machine Learning” and 1/2 the         topic “Artificial Intelligence”
    - Using each topic’s multinomial distribution, output words to fill the document’s word slots. In our example, the “Machine Learning”           topic is 1/2 our document. The “Machine Learning” topic might have the word “regression” at 20% probability or                                 “classification” at 15%, so it will fill the document word slots based on those probabilities.
- Parameters:
    - num_topics - The number of requested latent topics to be extracted from the training corpus.
    - passes - Number of learning iterations

In [64]:
start_time = time.time()
lda_20 = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary,passes=30,num_topics=10,random_state=200)
print("--- %s seconds ---" % (time.time() - start_time))
lda_20.save('lda_20.model')

--- 305.1613688468933 seconds ---


In [65]:
lda_20=gensim.models.ldamodel.LdaModel.load('lda_20.model')

In [67]:
#project.save_data('lda_20.model',gensim.models.ldamodel.LdaModel.load('lda_20.model'))

In [68]:
for i in lda_20.print_topics(num_topics=10,num_words=20):
    print("---------------------")
    print(i)
    print("---------------------")

---------------------
(0, '0.004*"work" + 0.004*"time" + 0.004*"users" + 0.003*"net_core" + 0.003*"device" + 0.003*"features" + 0.003*"like" + 0.003*"tools" + 0.003*"build" + 0.003*"apps" + 0.003*"apple" + 0.003*"thats" + 0.002*"support" + 0.002*"microsoft" + 0.002*"computer" + 0.002*"great" + 0.002*"product" + 0.002*"come" + 0.002*"working" + 0.002*"good"')
---------------------
---------------------
(1, '0.002*"time" + 0.002*"world" + 0.002*"history" + 0.001*"form" + 0.001*"said" + 0.001*"original" + 0.001*"life" + 0.001*"humans" + 0.001*"power" + 0.001*"known" + 0.001*"society" + 0.001*"says" + 0.001*"today" + 0.001*"project" + 0.001*"including" + 0.001*"found" + 0.001*"far" + 0.001*"great" + 0.001*"information" + 0.001*"french"')
---------------------
---------------------
(2, '0.004*"scientists" + 0.004*"said" + 0.004*"says" + 0.003*"et_al" + 0.003*"researchers" + 0.003*"research" + 0.003*"data" + 0.003*"ago_children" + 0.002*"study" + 0.002*"work" + 0.002*"patients" + 0.002*"poin

In [69]:
Data.shape

(6959, 12)

In [70]:
lda_20.show_topic(1,topn=20)

[('time', 0.0021027061),
 ('world', 0.0020487288),
 ('history', 0.0016113081),
 ('form', 0.0014841452),
 ('said', 0.0014335993),
 ('original', 0.0013324613),
 ('life', 0.001318223),
 ('humans', 0.0013079415),
 ('power', 0.0012189905),
 ('known', 0.0011892577),
 ('society', 0.0011567599),
 ('says', 0.0011448932),
 ('today', 0.0011425691),
 ('project', 0.0011178305),
 ('including', 0.0011115059),
 ('found', 0.0010945881),
 ('far', 0.001054376),
 ('great', 0.0010512116),
 ('information', 0.0010498712),
 ('french', 0.001022034)]

In [185]:
Words_All=[]
for i in range(10):
    words=[]
    topic=lda_20.show_topic(i,topn=20)
    for word in topic:
        words.append(word[0])
    Words_All.append(words)
#Words_All

In [199]:
import pickle
with open('words_from_topics.pkl', 'wb') as f:
    pickle.dump(Words_All, f)

In [190]:
Data.reset_index(inplace=True)

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            967, 968, 969, 970, 971, 972, 973, 974, 975, 976],
           dtype='int64', length=977)

In [200]:
vis = pyLDAvis.gensim.prepare(lda_20, corpus, dictionary)
pyLDAvis.display(vis)

In [201]:
pyLDAvis.save_html(vis, 'lda.html')